# Day 5

In [57]:
with open(f'data/test.txt') as f:
    lines = f.readlines()
    
    seeds = "".join(lines).split('\n\n')[0]
    seeds = [int(x) for x in seeds.split(':')[1].split()]
    
    maps = "".join(lines).split('\n\n')[1:]

In [40]:
maps

['seed-to-soil map:\n50 98 2\n52 50 48',
 'soil-to-fertilizer map:\n0 15 37\n37 52 2\n39 0 15',
 'fertilizer-to-water map:\n49 53 8\n0 11 42\n42 0 7\n57 7 4',
 'water-to-light map:\n88 18 7\n18 25 70',
 'light-to-temperature map:\n45 77 23\n81 45 19\n68 64 13',
 'temperature-to-humidity map:\n0 69 1\n1 0 69',
 'humidity-to-location map:\n60 56 37\n56 93 4']

In [31]:
for seed, loc in zip(seeds, [82, 43, 86, 35]):
    print(f'starting seed #{seed}')
    for index_map in maps:
        print(f'current seed #: {seed}')
#         print(index_map.split('\n')[1:])
#         print(len(index_map.split('\n')[1:]))
        for i, range_map in enumerate(index_map.split('\n')[1:]):
            dest, src, n = [int(x) for x in range_map.split()]
#             print(i)
#             print(dest, src, n)
            if src <= seed < (src + n):
                print('seed is in range')
                seed = (seed - src) + dest
                continue
                print(f'This is broken')
                
    print(f'Final location for seed is {seed}')
    assert seed == loc

starting seed #79
current seed #: 79
seed is in range
current seed #: 81
current seed #: 81
current seed #: 81
seed is in range
current seed #: 74
seed is in range
current seed #: 78
current seed #: 78
seed is in range
Final location for seed is 82
starting seed #14
current seed #: 14
current seed #: 14
seed is in range
current seed #: 53
seed is in range
seed is in range
current seed #: 38
seed is in range
current seed #: 31
current seed #: 31
seed is in range
current seed #: 32
Final location for seed is 32


AssertionError: 

So normally I write the preamble *before* I start trying anything, but this initially made no sense, so here we are after trying some shit.

What this problem comes down to is repeatedly mapping intervals of one range of numbers to another. My initial thought is that for each mapping action, we have to check if the seed's index is within the range that is being mapped. If it is, we map it to the new index and repeat for the following x-to-y maps.

The problem I have above is that in a single x-to-y mapping, once I find a range to map the input to, the new seed index might be in the next mapping interval so the seed gets mapped twice in a given map, which shouldn't happen. I need a way to break out interating in a given map for each seed pass

In [82]:
def map_function(inp, map_block):
    for map_interval in map_block:
        dest, src, n = [int(x) for x in map_interval.split()]
#         print(f'Current seed: {inp:,d}, map interval: {dest:,d}, {src:,d}, {n:,d}')
        
        if src <= inp < (src + n):
#             print(f"Found a match: {(inp - src):,d}")
            return (inp - src) + dest
    
    return inp

In [63]:
for seed, loc in zip(seeds, [82, 43, 86, 35]):
    print(f'starting seed #{seed}')
    for mapping in maps:
        seed = map_function(seed, mapping.split('\n')[1:])
    print(f'final seed location: {seed}')
    assert seed == loc

starting seed #929142010
final seed location: 2928341086


AssertionError: 

In [81]:
def map_function(inp, map_block):
    for map_interval in map_block:
        dest, src, n = [int(x) for x in map_interval.split()]
#         print(f'Current seed: {inp:,d}, map interval: {dest:,d}, {src:,d}, {n:,d}')
        
        if src <= inp < (src + n):
#             print(f"Found a match: {(inp - src):,d}")
            return (inp - src) + dest
    
    return inp

with open(f'data/input.txt') as f:
    lines = f.readlines()
    
    seeds = "".join(lines).split('\n\n')[0]
    seeds = [int(x) for x in seeds.split(':')[1].split()]
    
    maps = "".join(lines).split('\n\n')[1:]

final_locations = list()
for seed in seeds:
#     print(f'Starting seed: {seed}')
    for mapping in maps:
        map_name = mapping.split("\n")[0]
#         print(f'Starting map: {map_name}')
        map_string = mapping.split('\n')[1:]
        if map_string[-1] == '':
            map_string = map_string[:-1] # <- bane of my existence. File had an extra blank line at the end.... I probably should've just deleted in the file :/
#         print(f"current map: {map_string}")
        seed = map_function(seed, map_string)
    final_locations.append(seed)
    
print(f'Minimum location is: {sorted(final_locations)}')

Minimum location is: [600279879, 1249872241, 1377440429, 1471269794, 1787915678, 2064075662, 2290868489, 2306805090, 2418319456, 2743737607, 2760787031, 2768526595, 2928341086, 3135505386, 3322203336, 3651186410, 3872583593, 4011746621, 4013772529, 4017423988]


### Success! (finally)
---
## Puzzle 2

So now the input is modified such that ever pair of number is a range of seed indices. So now when we apply the mapping ranges, they might only partially overlap the current seed ranges

In [197]:
def map_function(inp, map_block):
    intervals = list()
    
    for map_interval in map_block:
#         print('Starting new map interval')
        dest, src, n = [int(x) for x in map_interval.split()]
#         print(f'Current seed: {inp:,d}, map interval: {dest:,d}, {src:,d}, {n:,d}')
        src_end = src+n
        dest_end = dest+n    
        delta = dest - src
        
#         print(inp, dest, src, n)
        unchanged = list()
        while inp:
            interval = inp.pop()
#             print(f'checking interval: {interval}')
            if interval[1] <= src or interval[0] > src_end:
#                 print(f'No Overlap')
                unchanged.append(interval)
                
            elif interval[0] < src and interval[1] < src_end:
#                 print(f'map starts in seeds')
                unchanged.append((interval[0], src))
                intervals.append((src+delta, interval[1]+delta))
#                 print(f'breaking up into {(interval[0], src)}, and {(src+delta, interval[1]+delta)}')
            elif interval[0] > src and interval[1] > src_end:
#                 print(f'seeds start in map')
                
                intervals.append((interval[0]+delta, src_end+delta))
                unchanged.append((src_end, interval[1]))

            elif interval[0] > src and interval[1] <= src_end:
#                 print(f'seed fully in map<<<<<<<<<<<<<<<<<<<<')
#                 print(f'map: {src}, {n} -> {dest}')
#                 print(f'seed interval: [{interval[0]}, {interval[1]}')
                seed_length = interval[1] - interval[0]
#                 print(f'seed interval length: {seed_length}')
#                 print(f'new interval: [{interval[0] + delta}, {interval[0] + delta + seed_length})')
                intervals.append((interval[0] + delta, interval[0] + delta + seed_length))
            else:
#                 print('map fully in seeds')
                if interval[0] != src:
                    unchanged.append((interval[0], src))
                intervals.append((src+delta, src+n+delta))
                if src+n != interval[1]:
                    unchanged.append((src+n, interval[1]))
                
#             print('\n')
#         print(f'Current seed intervals: {intervals}, {unchanged}')
        inp = unchanged

    return inp + intervals

In [201]:
with open(f'data/test.txt') as f:
    lines = f.readlines()
    
    seeds = "".join(lines).split('\n\n')[0]
    seeds = [int(x) for x in seeds.split(':')[1].split()]
    seeds = [(start, start+length) for start, length in zip(seeds[::2], seeds[1::2])]
    
    maps = "".join(lines).split('\n\n')[1:]

    locations = list()
    
    for seed in seeds:
#         print(f'starting new seed: {seed}***************')
        for map_block in maps:
#             print(f'starting new map block: {map_block}')
            if type(seed) != list:
                seed = [seed]
            seed = map_function(seed, map_block.split('\n')[1:])
        for loc_int in seed:
            locations.append(loc_int)
#             print(seed)
#         print(f'final seeds: {seed}')
#         print('\n')

In [203]:
locations

[(46, 56), (60, 61), (82, 85), (97, 99), (86, 90), (94, 97), (56, 60)]

In [210]:
with open(f'data/input.txt') as f:
    lines = f.readlines()
    
    seeds = "".join(lines).split('\n\n')[0]
    seeds = [int(x) for x in seeds.split(':')[1].split()]
    seeds = [(start, start+length) for start, length in zip(seeds[::2], seeds[1::2])]
    
    maps = "".join(lines).split('\n\n')[1:]

    locations = list()
    
    for seed in seeds:
#         print(f'starting new seed: {seed}***************')
        for map_block in maps:
#             print(f'starting new map block: {map_block}')
            
            if type(seed) != list:
                seed = [seed]
#             print(seed)
#             print(map_block.split('\n')[1:])
            seed = map_function(seed, map_block.split('\n')[1:])
        for loc_int in seed:
            locations.append(loc_int)
#             print(seed)
#         print(f'final seeds: {seed}')
#         print('\n')

In [212]:
!git add *
!git commit -m "Day 5 solution: I AM A GOLDEN GOD"
!git push -u origin main

(20191102, 23846357)

In [151]:
a

[(1, 2)]